In [1]:
%load_ext autoreload
%autoreload 2
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from site_health.users.models import User
from site_health.website.models import Site
from site_health.scraper.models import *
from site_health.metric.models import *
from site_health.scraper.tasks import *
user = User.objects.get(username='simon')
# page = Page.objects.all().last()
# site = Site.objects.get_or_create(url='https://www.simonnouwens.nl', owner=user)[0]
site = Site.objects.get_or_create(url='https://ok.nl', owner=user)[0]
# page = Page.objects.get_or_create(site=site, url_part='')

In [27]:
site = Site.objects.get_or_create(url='https://bogatech.nl', owner=user)

In [26]:
site[0].delete()

In [21]:
tdiff_seconds = 500
import numpy as np
np.random.randint(
            np.round(tdiff_seconds * 0.7), np.round(tdiff_seconds * 1.3)
        )

601

In [11]:
scraper = PageScraper.objects.get(id='b384faa9-8717-46cb-b34d-ba3cca1abf1d')

In [12]:
data = scraper.run()

In [16]:
scraper.get_html_processor().extract_urls_to_self()

['https://bogatech.nl/',
 'https://bogatech.nl/project/',
 'https://bogatech.nl/catalogus/',
 'https://bogatech.nl/project/buizen-en-verbindingen-volgens-din/',
 'https://bogatech.nl/project/buisophanging-isolatie-en-toebehoren/',
 'https://bogatech.nl/project/kunststof-leidingssystemen-en-toebehoren/',
 'https://bogatech.nl/project/bevestigingsmaterialen/',
 'https://bogatech.nl/project/kleine-ijzerwaren/',
 'https://bogatech.nl/project/elektrisch-gereedschap/',
 'https://bogatech.nl/project/handgereedschap/',
 'https://bogatech.nl/project/kleding-en-veiligheid/',
 'https://bogatech.nl/project/schoonmaakmiddelen/',
 'https://bogatech.nl/project/afdichtingskitten-en-lijmen/',
 'https://bogatech.nl/project/chemische-producten/',
 'https://bogatech.nl/project/electro-installatie/',
 'https://bogatech.nl/project/verven',
 'https://bogatech.nl/contact/',
 'https://bogatech.nl/catalogus',
 'https://bogatech.nl/project/verven/',
 'https://bogatech.nl/project/schoonmaakartikelen/',
 'https://

In [23]:
str(timezone.now())

'2020-06-29 11:28:07.240905+00:00'

In [14]:
link[: len(str(processor.page.site.url))]
str(processor.page.site.url)    

'https://www.ok.nl'

In [25]:
run_scraper.delay('ssl', '731ad7bb-7eb8-4ee1-ac09-ef61c2aa35f1')

<AsyncResult: b52f13fc-b398-4726-a3c6-d11cefb72bc4>

In [41]:
scraper = GetScraper().factory(page=site.page_set.all()[0])

In [3]:
RedirectMetrics().factory(site.page_set.all()[0]).run()

In [22]:
scraper = PageBrowserScraper().factory(site.page_set.all()[0])
scraper.wait_ms_before_screenshot = 500
scraper.take_screenshot=True
scraper.save()

In [23]:
selenium_scraper_execute.delay(scraper.id)

<AsyncResult: a48ff57a-49b5-4d14-9d1f-a428783b004c>

In [6]:
Site.objects.all()[1].page_set.all()

<QuerySet [<Page: https://www.ok.nl>]>

In [31]:
from celery.task.control import inspect

In [15]:
import tld

In [40]:
res = tld.get_tld('https://ab.dgdg.ok.nl/adgihiadhg?asfin=eave', as_object=True)

In [41]:
dir(res)

['_Result__fld',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 'domain',
 'extension',
 'fld',
 'parsed_url',
 'subdomain',
 'suffix',
 'tld']

'ab.dgdg.ok.nl'